# Text Preprocessing for Reddit Comment Classification

Outstanding TODOs:
* incorporate meta-feature and POS tags into feature list
* run tfidf with more computing power so that n-grams can be increased to 2 or 3

In [1]:
import numpy as np
import scipy
import pandas as pd
import nltk
import string
from nltk.tokenize import TweetTokenizer
from nltk.tag import pos_tag_sents
from nltk.stem import WordNetLemmatizer
from spellchecker import SpellChecker
from collections import Counter
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from pymfe.mfe import MFE

comment_data = pd.read_csv('../Data/reddit_train.csv')
test_data = pd.read_csv('../Data/reddit_test.csv')
print(comment_data.head())
print(test_data.head())

   id                                           comments       subreddits
0   0  Honestly, Buffalo is the correct answer. I rem...           hockey
1   1  Ah yes way could have been :( remember when he...              nba
2   2  https://youtu.be/6xxbBR8iSZ0?t=40m49s\n\nIf yo...  leagueoflegends
3   3  He wouldn't have been a bad signing if we woul...           soccer
4   4  Easy. You use the piss and dry technique. Let ...            funny
   id                                           comments
0   0  Trout and Bryant have both led the league in s...
1   1  &gt; Just like Estonians have good reasons to ...
2   2  Will Sol_Primeval sotp being oblivious?\n\nfin...
3   3  Moving Ostwald borders back to the pre 1967 bo...
4   4         You have to take it out of the bag, Morty!


### Cleanup of Input Data
Standardize the text snippets of the comments, preparing the comments for further analysis.

In [2]:
# punctuation removal
comment_data['prep'] = comment_data['comments'].str.replace(r'[^\w\s]+', '')
test_data['prep'] = test_data['comments'].str.replace(r'[^\w\s]+', '')

# lowercase
comment_data['prep'] = comment_data['prep'].str.lower()
test_data['prep'] = test_data['prep'].str.lower()

# convert numbers to 'num'
comment_data['prep'] = comment_data['prep'].str.replace('(\d+)', ' num ')
test_data['prep'] = test_data['prep'].str.replace('(\d+)', ' num ')

# replace links with 'wasurl'
comment_data['prep'] = comment_data['prep'].str.replace(r'http(?<=http).*', ' wasurl ')
test_data['prep'] = test_data['prep'].str.replace(r'http(?<=http).*', ' wasurl ')

# replace newlines and tabs with spaces
comment_data['prep'] = comment_data['prep'].str.replace(r'\s+', " ")
test_data['prep'] = test_data['prep'].str.replace(r'\s+', " ")

# fix any double spaces we created in the previous steps
comment_data['prep'] = comment_data['prep'].str.replace(" +", " ")
test_data['prep'] = test_data['prep'].str.replace(" +", " ")

# # typo correction (TODO commented out because it takes too long)
# spell = SpellChecker()
# def spellcheck_col(row):
#     return [spell.correction(word) for word in row]
# comment_data['prep'] = comment_data.prep.apply(spellcheck_col)
# test_data['prep'] = test_data.prep.apply(spellcheck_col)

# lemmatization
lemmatizer = WordNetLemmatizer()
tt = TweetTokenizer()
def lemmatize_col(row):
    row = tt.tokenize(row)
    return ' '.join([lemmatizer.lemmatize(w) for w in row])
comment_data['prep'] = comment_data.prep.apply(lemmatize_col)
test_data['prep'] = test_data.prep.apply(lemmatize_col)

print(comment_data)
print(test_data)

          id                                           comments  \
0          0  Honestly, Buffalo is the correct answer. I rem...   
1          1  Ah yes way could have been :( remember when he...   
2          2  https://youtu.be/6xxbBR8iSZ0?t=40m49s\n\nIf yo...   
3          3  He wouldn't have been a bad signing if we woul...   
4          4  Easy. You use the piss and dry technique. Let ...   
...      ...                                                ...   
69995  69995  Thank you, you confirm Spain does have nice pe...   
69996  69996  Imagine how many he would have killed with a r...   
69997  69997  Yes. Only. As in the guy I was replying to was...   
69998  69998  Looking for something light-hearted or has a v...   
69999  69999  I love how I never cry about casters because I...   

            subreddits                                               prep  
0               hockey  honestly buffalo is the correct answer i remem...  
1                  nba  ah yes way could ha

### Part-of-Speech Count, TF-IDF (with stopword removal), and Meta-Feature Extraction
Use POS tagging, TF-IDF, and meta-feature extraction to generate feature vectors.

In [3]:
# meta-feature extraction
def avg_word(sentence):
    words = sentence.split()
    return (sum(len(word) for word in words)/len(words))

# word count
comment_data['word_count'] = comment_data['comments'].apply(lambda x: len(str(x).split(" ")))
test_data['word_count'] = comment_data['comments'].apply(lambda x: len(str(x).split(" ")))

# character count
comment_data['char_count'] = comment_data['comments'].str.len()
test_data['char_count'] = test_data['comments'].str.len()

# average word length
comment_data['avg_word'] = comment_data['comments'].apply(lambda x: avg_word(x))
test_data['avg_word'] = test_data['comments'].apply(lambda x: avg_word(x))

# stopword count (stopwords will be removed later)
stopwords = nltk.corpus.stopwords.words('english')
comment_data['stop_count'] = comment_data['comments'].apply(lambda x: len([x for x in x.split() if x in stopwords]))
test_data['stop_count'] = test_data['comments'].apply(lambda x: len([x for x in x.split() if x in stopwords]))

# digit count
comment_data['digit_count'] = comment_data['comments'].apply(lambda x: len([x for x in x.split() if x.isdigit()]))
test_data['digit_count'] = test_data['comments'].apply(lambda x: len([x for x in x.split() if x.isdigit()]))

# mfe = MFE(groups=["general", "statistical", "info-theory"], summary=["min", "median", "max"])
# # TODO remove .head(1000) (currently included because memory limit doesn't allow computation of full list)
# mfe.fit(comment_data['comments'].head(1000).tolist(), comment_data['subreddits'].head(1000).tolist())
# ft = mfe.extract()
# print(ft)

print(comment_data.head())
print(test_data.head())

   id                                           comments       subreddits  \
0   0  Honestly, Buffalo is the correct answer. I rem...           hockey   
1   1  Ah yes way could have been :( remember when he...              nba   
2   2  https://youtu.be/6xxbBR8iSZ0?t=40m49s\n\nIf yo...  leagueoflegends   
3   3  He wouldn't have been a bad signing if we woul...           soccer   
4   4  Easy. You use the piss and dry technique. Let ...            funny   

                                                prep  word_count  char_count  \
0  honestly buffalo is the correct answer i remem...          58         357   
1  ah yes way could have been remember when he wa...          29         145   
2  wasurl if you didnt find it already nothing ou...          18         145   
3  he wouldnt have been a bad signing if we would...          24         123   
4  easy you use the piss and dry technique let a ...          46         212   

   avg_word  stop_count  digit_count  
0  5.245614      

In [4]:
# part-of-speech tagging
# comment_data['comments_tagged'] = pos_tag_sents(comment_data['prep'].tolist())
# test_data['comments_tagged'] = pos_tag_sents(test_data['prep'].tolist())
# TODO count totals of each part of speech (noun, adjective, etc) and use the counts as features

# print(comment_data.head())
# print(test_data.head())

In [5]:
# in the bag of words matrix, remove punctuation and stopwords
# count_vectorizer = CountVectorizer(tokenizer=tt.tokenize, stop_words="english", ngram_range=(1,3))
# counts = count_vectorizer.fit_transform(comment_data.comments)
# print("raw word count matrix size: " + str(counts.shape))
# count_t_vectorizer = CountVectorizer(tokenizer=tt.tokenize, stop_words="english", ngram_range=(1,3))
# counts_t = count_t_vectorizer.fit_transform(test_data.comments)
# print("raw word count matrix size: " + str(counts_t.shape))

# TF-IDF, also removing punctuation and stopwords
tfidf_vectorizer = TfidfVectorizer(tokenizer=tt.tokenize, stop_words="english", ngram_range=(1,2))
tfidf = tfidf_vectorizer.fit_transform(comment_data.prep)
print("TF-IDF matrix size: " + str(tfidf.shape))
tfidf_t = tfidf_vectorizer.transform(test_data.prep)
print("TF-IDF_t matrix size: " + str(tfidf_t.shape))

TF-IDF matrix size: (70000, 927355)
TF-IDF_t matrix size: (30000, 927355)


In [6]:
# append extra features to TFIDF matrix
c_wc = scipy.sparse.csr_matrix.transpose(scipy.sparse.csr_matrix(comment_data['word_count'].to_numpy()))
# c_cc = scipy.sparse.csr_matrix.transpose(scipy.sparse.csr_matrix(comment_data['char_count'].to_numpy()))
c_aw = scipy.sparse.csr_matrix.transpose(scipy.sparse.csr_matrix(comment_data['avg_word'].to_numpy()))
c_sc = scipy.sparse.csr_matrix.transpose(scipy.sparse.csr_matrix(comment_data['stop_count'].to_numpy()))
c_dc = scipy.sparse.csr_matrix.transpose(scipy.sparse.csr_matrix(comment_data['digit_count'].to_numpy()))
t_wc = scipy.sparse.csr_matrix.transpose(scipy.sparse.csr_matrix(test_data['word_count'].to_numpy()))
# t_cc = scipy.sparse.csr_matrix.transpose(scipy.sparse.csr_matrix(test_data['char_count'].to_numpy()))
t_aw = scipy.sparse.csr_matrix.transpose(scipy.sparse.csr_matrix(test_data['avg_word'].to_numpy()))
t_sc = scipy.sparse.csr_matrix.transpose(scipy.sparse.csr_matrix(test_data['stop_count'].to_numpy()))
t_dc = scipy.sparse.csr_matrix.transpose(scipy.sparse.csr_matrix(test_data['digit_count'].to_numpy()))

# feature_matrix = scipy.sparse.hstack((tfidf, c_wc, c_cc, c_aw, c_sc, c_dc))
# feature_matrix_t = scipy.sparse.hstack((tfidf_t, t_wc, t_cc, t_aw, t_sc, t_dc))
feature_matrix = scipy.sparse.hstack((tfidf, c_wc, c_aw, c_sc, c_dc))
feature_matrix_t = scipy.sparse.hstack((tfidf_t, t_wc, t_aw, t_sc, t_dc))

# TODO fix NaN values in c_cc and t_cc

print("Training matrix size: " + str(feature_matrix.shape))
print("Test matrix size: " + str(feature_matrix_t.shape))

Training matrix size: (70000, 927359)
Test matrix size: (30000, 927359)


#### File Export
Dump the vectorizer, feature matrices, and dataframes to a pickle file in order for other group members to use them.

In [7]:
import pickle

# vectorizer
with open('../Data/vectorizer.pk', 'wb') as file:
    pickle.dump(tfidf_vectorizer, file)

# feature matrices (load on the other side with scipy.sparse.load_npz())
scipy.sparse.save_npz('../Data/feature_matrix_train.npz', feature_matrix)
scipy.sparse.save_npz('../Data/feature_matrix_test.npz', feature_matrix_t)

# dataframes of comment data
cd = comment_data.to_numpy()
print(cd.shape)
np.save('../Data/cleaned_data_train.npy', comment_data.to_numpy())
np.save('../Data/cleaned_data_test.npy', test_data.to_numpy())

(70000, 9)


### Feature Selection
Pick the important information out of the large matrix generated above. Only necessary if the number of features is so high that the models don't have enough computation power to train, OR when some features are providing poor information and throwing off results.

In [8]:
# # perform PCA on the TF-IDF matrix
# from sklearn.decomposition import TruncatedSVD

# tsvd = TruncatedSVD(n_components=20)
# reduced_features = tsvd.fit_transform(tfidf)
# print(tsvd.explained_variance_ratio_)

In [9]:
# from sklearn.feature_selection import chi2

# N = 2
# for Product, category_id in sorted(category_to_id.items()):
#   features_chi2 = chi2(tfidf, labels == category_id)
#   indices = np.argsort(features_chi2[0])
#   feature_names = np.array(tfidf.get_feature_names())[indices]
#   unigrams = [v for v in feature_names if len(v.split(' ')) == 1]
#   bigrams = [v for v in feature_names if len(v.split(' ')) == 2]
#   print("# '{}':".format(Product))
#   print("  . Most correlated unigrams:\n. {}".format('\n. '.join(unigrams[-N:])))
#   print("  . Most correlated bigrams:\n. {}".format('\n. '.join(bigrams[-N:])))

### Models
SciKit-Learn implementations of decision tree and Multinomial Naive Bayes.

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.ensemble import RandomForestClassifier

# decision tree on test set
d_tree = DecisionTreeClassifier(random_state=0).fit(feature_matrix, comment_data['subreddits'])
d_tree_predict = d_tree.predict(feature_matrix_t)
print("Decision Tree test set predictions:")
print(d_tree_predict)

# niave bayes on test set
nb = MultinomialNB().fit(feature_matrix, comment_data['subreddits'])
nb_predict = nb.predict(feature_matrix_t)
print("Naive Bayes test set predictions:")
print(nb_predict)

# random forest classifier on test set
rf = RandomForestClassifier().fit(feature_matrix, comment_data['subreddits'])
rf_predict = rf.predict(feature_matrix_t)
print("Random Forest test set predictions:")
print(rf_predict)

# check accuracy using same training set values (overfitting, this is just a quick-and-dirty check)
sanity_score = d_tree.score(feature_matrix, comment_data['subreddits'])
print("Sanity check - decision tree accuracy on training set is: " + str(sanity_score))


#### TODO run the train-test split data through proper pre-processing steps ####
# make a validation set to properly check accuracy
X_train, X_val, y_train, y_val = train_test_split(comment_data['comments'], comment_data['subreddits'], random_state = 0)
x_train_features = tfidf_vectorizer.transform(X_train)
x_val_features = tfidf_vectorizer.transform(X_val)

d_tree_val = DecisionTreeClassifier(random_state=0).fit(x_train_features, y_train)
d_tree_score = d_tree_val.score(x_val_features, y_val)
print("Decision tree validation score: " + str(d_tree_score))

nb_val = MultinomialNB().fit(x_train_features, y_train)
nb_score = nb_val.score(x_val_features, y_val)
print("Naive Bayes validation score: " + str(nb_score))

rf_val = RandomForestClassifier().fit(x_train_features, y_train)
rf_score = rf_val.score(x_val_features, y_val)
print("Random Forest validation score: " + str(rf_score))

Decision Tree test set predictions:
['soccer' 'worldnews' 'movies' ... 'Overwatch' 'gameofthrones' 'wow']
Naive Bayes test set predictions:
['Music' 'Music' 'Music' ... 'Music' 'Music' 'Music']


### One-Hot Encoding of Classifiers
To be used for the models implemented from scratch.

In [ ]:
# categories
labels = ['hockey', 'nba', 'leagueoflegends', 'soccer', \
          'funny', 'movies', 'anime', 'Overwatch' 'trees', \
          'GlobalOffensive', 'nfl', 'AskReddit', 'gameofthrones', \
          'conspiracy', 'worldnews', 'wow', 'europe', 'canada', \
          'Music', 'baseball']


# default setup
full_reference = np.eye(len(labels))
partial_reference = np.append(np.zeros((1,len(labels)-1), dtype=np.int8),\
                              np.eye(len(labels)-1, dtype=np.int8), axis = 0)

# encoder
def encode(label, labels=labels, ref=full_reference):
    location = labels.index(label)
    return ref[location]


# shapes
print(full_reference.shape)
print(partial_reference.shape)

# example
print(encode('hockey'))